In [464]:
#TODO nao estou tratanto o caso do OR
#TODO nao nao trado caso o usario queira colocar um intervalor tipo assim 30<=var<=40

In [465]:
import json
import pandas as pd
from pyparsing import Word, alphas, nums, oneOf, infixNotation, opAssoc, ParserElement
from sympy import sympify
from sympy.logic.boolalg import to_dnf
import re
# Configurações para exibir todas as colunas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [466]:
def get_desired_scenario(file_path = 'analyzed_scenario_process.json'):
    # Ler o arquivo JSON
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Pegar o valor da chave 'desired'
    desired_scenario = data.get('desired')
    # desired_scenario_name = next(iter(desired_scenario))
    # Exibir o valor

    # print("Desired scenario: ", desired_scenario)
    
    return desired_scenario

# get_desired_scenario()

In [467]:
def load_variable_ranges(file_path="variables_config.json"):
    with open(file_path, 'r') as file:
        json_data = file.read()
    variable_ranges = json.loads(json_data)
    
    # print("Variable ranges: ", {var['name']: var for var in variable_ranges})

    return {var['name']: var for var in variable_ranges}

# load_variable_ranges()

In [468]:
#TODO, VER SE VAI SER NECESSARIO FAZER ALGUMA ANALISE DE ERRO ENTRE A POSTCONDICAO DO CENÁRIO RECOMENDADO COM O CONTEXTO ALCANÇADO
# def get_shared_scenarios(file_path = 'shared_base.json'):
#     # Ler o arquivo JSON
#     with open(file_path, 'r') as file:
#         shared_scenarios = json.load(file)
#     return shared_scenarios

In [469]:
def get_Then(scenario):    
    scenario_name = next(iter(scenario))
    then_exp=scenario[scenario_name]["Then"]
    # print("Then: ", then_exp)
    return then_exp

# print(get_Then(get_desired_scenario()))

In [470]:
def logical_expression_settings():
    # Configurar pyparsing para ignorar espaços em branco
    ParserElement.enablePackrat()

    # Definir elementos básicos da expressão
    identifier = Word(alphas, alphas + nums + "_")
    number = Word(nums)

    # Definir operadores de comparação
    comparison_operator = oneOf("<= >= < > == !=")

    # Definir expressão básica
    operand = identifier | number

    # Definir expressão lógica
    logical_expr = infixNotation(operand,
        [
            (comparison_operator, 2, opAssoc.LEFT),
            ("AND", 2, opAssoc.LEFT),
            ("OR", 2, opAssoc.LEFT),
        ])

    return logical_expr


def extract_leaves(parsed_expr):
    """Extrai folhas de uma expressão lógica analisada."""
    
    if isinstance(parsed_expr, str):
        return []
    elif len(parsed_expr) == 3 and parsed_expr[1] in ["<=", ">=", "<", ">", "==", "!="]:
        return [" ".join(parsed_expr)]
    else:
        leaves = []
        for sub_expr in parsed_expr:
            leaves.extend(extract_leaves(sub_expr))
        return leaves

# Exemplo de uso
# expression_scenario = "(((a <= 3 AND (b > 10 OR c == 10)) OR ((d != 5 AND e >= 20) OR f < 2)) AND ((g <= 7 OR h > 8) AND (i == 9 OR j != 3)))"
expression_scenario = "(a <= 3 AND b > 10)"

# expression_scenario = get_Given(get_desired_scenario())

parsed_expression = logical_expression_settings().parseString(expression_scenario, parseAll=True)

print("Extracted Leafs:", extract_leaves(parsed_expression.asList()))


Extracted Leafs: ['a <= 3', 'b > 10']


In [471]:
def list_to_indexed_dict(lst):
    """
    Transforms a list of strings into a dictionary with keys 'x1', 'x2', ..., 'xn'.
    
    Parameters:
    lst (list): List of strings.

    Returns:
    dict: Dictionary with keys 'x1', 'x2', ..., 'xn' and values from the original list.
    """
    return {f'x{i+1}': value for i, value in enumerate(lst)}

# Example usage
lst = ['a <= 3', 'b > 10']
map_dict = list_to_indexed_dict(extract_leaves(parsed_expression.asList()))
print("Indexed Dict:", map_dict)


Indexed Dict: {'x1': 'a <= 3', 'x2': 'b > 10'}


In [472]:
def replace_values_with_keys(string, dictionary):
    """
    Replaces occurrences of the dictionary values with the keys in the string.

    Parameters:
    string (str): The string where the replacements will be made.
    dictionary (dict): The dictionary with keys and values to be replaced.

    Returns:
    str: The string with values replaced by keys.
    """
    for key, value in dictionary.items():
        string = string.replace(value, key)
    return string

indexed_scenario = replace_values_with_keys(expression_scenario, map_dict)
print(indexed_scenario)


(x1 AND x2)


In [473]:
def replace_logical_operators(string, scenario_operators=False):
    """
    Substitui os operadores lógicos 'AND' por '&' e 'OR' por '|', ou vice-versa.

    Parâmetros:
    string (str): A string onde as substituições serão feitas.
    scenario_operators (bool): Se True, substitui '&' por 'AND' e '|' por 'OR'.
                               Se False, substitui 'AND' por '&' e 'OR' por '|'.

    Retorno:
    str: A string com os operadores lógicos substituídos.
    """
    if scenario_operators:
        return string.replace('&', 'AND').replace('|', 'OR')
    else:
        return string.replace('AND', '&').replace('OR', '|')



indexed_scenario = replace_logical_operators(indexed_scenario)
print("indexed_scenario:", indexed_scenario)

indexed_scenario: (x1 & x2)


In [474]:
def _to_dnf(indexed_scenario):
    """
    Converts a logical expression to Disjunctive Normal Form (DNF).

    Parameters:
    indexed_scenario (str): The logical expression in string format with indexed variables.

    Returns:
    str: The logical expression in Disjunctive Normal Form.
    """
    # Convert the string to a symbolic expression
    logical_expression = sympify(indexed_scenario)

    # Convert the symbolic expression to Disjunctive Normal Form
    dnf_expr = to_dnf(logical_expression, simplify=True, force=True)

    return str(dnf_expr)

# Example usage
dnf_expr = _to_dnf(indexed_scenario)
print(dnf_expr)

x1 & x2


In [475]:
def revert_keys_to_values(string, dictionary):
    """
    Replaces occurrences of the dictionary keys with the values in the string.

    Parameters:
    string (str): The string where the replacements will be made.
    dictionary (dict): The dictionary with keys and values to be replaced.

    Returns:
    str: The string with the keys replaced by the original values.
    """
    for key, value in dictionary.items():
        string = string.replace(key, value)
    
    return string

# Example usage
final_string = revert_keys_to_values(str(dnf_expr), map_dict)
print(f"Final string with original values: {final_string}")


Final string with original values: a <= 3 & b > 10


In [476]:
def to_dnf_expression(expression_scenario):

    parsed_expression = logical_expression_settings().parseString(expression_scenario, parseAll=True)
    # print("Parsed Expression:", parsed_expression)

    extracted_leaves=extract_leaves(parsed_expression.asList())
    # print("Extracted Leafs:", extracted_leaves)

    map_dict = list_to_indexed_dict(extracted_leaves)
    # print("Indexed Dict:", map_dict)

    indexed_scenario = replace_values_with_keys(expression_scenario, map_dict)
    # print("replaced values with keys:", indexed_scenario)

    indexed_scenario = replace_logical_operators(indexed_scenario)
    # print("indexed_scenario:", indexed_scenario)

    dnf_expr = _to_dnf(indexed_scenario)
    # print(dnf_expr)

    # Example usage
    dnf_expression = revert_keys_to_values(str(dnf_expr), map_dict)
    # print(f"Final string with original values: {dnf_expression}")

    dnf_expression=replace_logical_operators(dnf_expression, scenario_operators=True)
    # print(f"dnf_expression: {dnf_expression}")

    return dnf_expression

In [477]:
def parse_expression(leaf):
    """
    Parse the expression into its components: variable, operator, and value.
    """
    match = re.match(r"(\w+)\s*(>=|<=|>|<|==|!=)\s*(.+)", leaf)
    if match:
        variable, operator, value = match.groups()
        return variable, operator, value
    else:
        raise ValueError(f"Invalid expression format: {leaf}")

def get_expression_range(var_name, operator, value, min_val, max_val):
    """
    Determine the range of values that satisfy the expression.
    """
    value = float(value)
    if operator == '>=':
        return (value, max_val)
    elif operator == '<=':
        return (min_val, value)
    elif operator == '>':
        return (value + 1e-9, max_val)  # Adding a small value to make it exclusive
    elif operator == '<':
        return (min_val, value - 1e-9)  # Subtracting a small value to make it exclusive
    elif operator == '==':
        return (value, value)
    elif operator == '!=':
        return [(min_val, value - 1e-9), (value + 1e-9, max_val)]  # Returns two ranges
    else:
        raise ValueError(f"Unsupported operator: {operator}")

def calculate_jaccard_similarity(leaf1, leaf2, variable_ranges):
    """
    Calculate the Jaccard similarity between two expressions.
    The expressions should be of the format: "variable operator value".
    """
    try:
        var1, op1, val1 = parse_expression(leaf1)
        var2, op2, val2 = parse_expression(leaf2)
    except ValueError as e:
        print(e)
        return 0.0

    # Ensure the variable names are the same
    if var1 != var2:
        return 0.0

    # Get variable ranges
    var_info = variable_ranges.get(var1)
    if not var_info:
        print(f"Variable {var1} not found in ranges.")
        return 0.0

    min_val = var_info['min_value']
    max_val = var_info['max_value']

    # Get ranges for each expression
    range1 = get_expression_range(var1, op1, val1, min_val, max_val)
    range2 = get_expression_range(var2, op2, val2, min_val, max_val)

    def calculate_intersection_union(range1, range2):
        """
        Helper function to calculate intersection and union of two ranges.
        """
        if isinstance(range1, list):
            intersections = []
            for r1 in range1:
                if isinstance(range2, list):
                    for r2 in range2:
                        intersections.append((max(r1[0], r2[0]), min(r1[1], r2[1])))
                else:
                    intersections.append((max(r1[0], range2[0]), min(r1[1], range2[1])))
        else:
            if isinstance(range2, list):
                intersections = [(max(range1[0], r2[0]), min(range1[1], r2[1])) for r2 in range2]
            else:
                intersections = [(max(range1[0], range2[0]), min(range1[1], range2[1]))]
        
        valid_intersections = [r for r in intersections if r[0] <= r[1]]
        intersection_length = sum(r[1] - r[0] for r in valid_intersections)
        
        union_min = min(range1[0] if not isinstance(range1, list) else min(r[0] for r in range1),
                        range2[0] if not isinstance(range2, list) else min(r[0] for r in range2))
        union_max = max(range1[1] if not isinstance(range1, list) else max(r[1] for r in range1),
                        range2[1] if not isinstance(range2, list) else max(r[1] for r in range2))
        union_length = union_max - union_min
        
        return intersection_length, union_length

    # Calculate intersection and union considering the equality operator cases
    if op1 == '==' and op2 != '==':
        if isinstance(range2, list):
            range2_min, range2_max = min(range2[0][0], range2[1][0]), max(range2[0][1], range2[1][1])
            if range2_min <= range1[0] <= range2_max:
                intersection_length = 1
                union_length = range2_max - range2_min
            else:
                return 0.0
        else:
            if range2[0] <= range1[0] <= range2[1]:
                intersection_length = 1
                union_length = range2[1] - range2[0]
            else:
                return 0.0
    elif op2 == '==' and op1 != '==':
        if isinstance(range1, list):
            range1_min, range1_max = min(range1[0][0], range1[1][0]), max(range1[0][1], range1[1][1])
            if range1_min <= range2[0] <= range1_max:
                intersection_length = 1
                union_length = range1_max - range1_min
            else:
                return 0.0
        else:
            if range1[0] <= range2[0] <= range1[1]:
                intersection_length = 1
                union_length = range1[1] - range1[0]
            else:
                return 0.0
    elif op1 == '==' and op2 == '==':
        if val1 == val2:
            return 1.0
        else:
            return 0.0
    else:
        intersection_length, union_length = calculate_intersection_union(range1, range2)

    # Calculate the Jaccard similarity
    similarity = intersection_length / union_length
    return round(similarity,5)

# Load the variable ranges from the file
# variable_ranges_from_file = load_variable_ranges()

# Test cases
# test_cases = [
#     ("height == 30", "height <= 40"),  # equality with range
#     ("height == 30", "height == 30"),  # equality with equality (same)
#     ("height == 30", "height == 40"),  # equality with equality (different)
#     ("height >= 30", "height <= 40"),  # range with range
#     ("height != 30", "height <= 40"),  # inequality with range
#     ("height != 30", "height == 40")   # inequality with equality
# ]

# var_similarity_dict = {f"{expr1} vs {expr2}": calculate_jaccard_similarity(expr1, expr2, variable_ranges_from_file)
#            for expr1, expr2 in test_cases}

# # # Convert results to DataFrame and display
# # results_df = pd.DataFrame(results.items(), columns=["Test Case", "Similarity"])
# # results_df
# var_similarity_dict




In [478]:
# calculate_jaccard_similarity('a > 10', 'a == 12', load_variable_ranges())

In [479]:
def local_similarity(leaf1, leaf2, variable_ranges_from_file):
    local_similarity = calculate_jaccard_similarity(leaf1, leaf2, variable_ranges_from_file)
    # print(f"Local similarity between {leaf1} vs {leaf2}: {local_similarity}")
    return local_similarity

In [480]:
def get_leafs(dnf):
    parsed_expression = logical_expression_settings().parseString(dnf, parseAll=True)
    # print("Parsed Expression:", parsed_expression)

    extracted_leaves=extract_leaves(parsed_expression.asList())
    # print("Extracted Leafs:", extracted_leaves)
    return extracted_leaves

In [481]:
def extract_variables(leafts):
    """
    Extracts the variable names from a list of leaf strings.

    Parameters:
    leaves (list): List of strings, each containing a variable, an operator, and a value.

    Returns:
    list: List of variable names.
    """
    variables = []
    for leaf in leafts:
        # Using regular expression to match the variable name at the beginning of the string
        match = re.match(r"([a-zA-Z_]\w*)\s*(==|!=|[<>]=?)\s*\d+", leaf)
        if match:
            variables.append(match.group(1))
    return variables

In [482]:
def tversky_similarity(vars1, vars2, alpha=1.0, beta=1.0):
    """
    Calculate the Tversky similarity between two sets of variables.
    
    Args:
    vars1 (list): First list of variables.
    vars2 (list): Second list of variables.
    alpha (float): Weight for the elements unique to vars1. Default is 1.0.
    beta (float): Weight for the elements unique to vars2. Default is 1.0.
    
    Returns:
    float: The Tversky similarity score.
    """
    set1 = set(vars1)
    set2 = set(vars2)
    
    intersection = len(set1 & set2)
    only_in_set1 = len(set1 - set2)
    only_in_set2 = len(set2 - set1)
    
    tversky_index = intersection / (intersection + alpha * only_in_set1 + beta * only_in_set2)
    
    return tversky_index

def penality(vars1, vars2, alpha=1.0, beta=1.0):
    return 1 - tversky_similarity(vars1, vars2, alpha, beta)


In [483]:
def intersection(vector1, vector2):
    """
    Find the intersection between two vectors of elements.
    
    Args:
    vector1 (list): First list of elements.
    vector2 (list): Second list of elements.
    
    Returns:
    list: A list containing the elements that are present in both vectors.
    """
    set1 = set(vector1)
    set2 = set(vector2)
    
    intersection = set1 & set2
    
    return list(intersection)


In [484]:
def pair_leaves(leaves1, leaves2):
    """
    Create pairs of leaves containing the same variable.
    
    Args:
    leaves1 (list): First list of leaves.
    leaves2 (list): Second list of leaves.
    
    Returns:
    list: A list of tuples, each containing a pair of leaves with the same variable.
    """
    pairs = []
    
    # Extract variable names from leaves
    def extract_variable(leaf):
        return leaf.split()[0]
    
    variables1 = {extract_variable(leaf): leaf for leaf in leaves1}
    variables2 = {extract_variable(leaf): leaf for leaf in leaves2}
    
    # Create pairs
    for var in variables1:
        if var in variables2:
            pairs.append((variables1[var], variables2[var]))
    
    return pairs


In [485]:
# funcao Mocked para similar os pesos.
def get_weights():
    # TODO: Implementar a lógica para obter os pesos com esse formato {'a': 0.75, 'b': 0.0, 'c': 0.0}
    return {} # Weight is considered 1 when variable is not present in the weights dictionary

weight_dict = get_weights()
# print("Weights:{}".format(weight_dict))

In [486]:
def extract_variable_similarity(local_similarity):
    """
    Extract variable names and their similarities from a dictionary of leaf pairs.
    
    Args:
    pairs_similarity (dict): A dictionary with tuples of leaf pairs as keys and similarity values as values.
    
    Returns:
    dict: A dictionary with variable names as keys and similarity values as values.
    """
    variable_similarity = {}
    
    for (leaf1, leaf2), similarity in local_similarity.items():
        variable = leaf1.split()[0]
        variable_similarity[variable] = similarity
    
    return variable_similarity

In [487]:
def global_satisfaction(local_similarity):
    weight_dict=get_weights()
    variable_similarity_dict=extract_variable_similarity(local_similarity)

    weighted_sum = 0
    total_weight = 0
    
    for var, error_obj in variable_similarity_dict.items():
        error_value = error_obj["error"]
        weight = weight_dict.get(var, 1)
        weighted_sum += error_value * weight
        total_weight += weight

    if total_weight == 0:
        return 0  # Avoid division by zero
    
    weighted_average = weighted_sum / total_weight
    return 1-weighted_average

In [488]:
def scenario_similarity(final_similarity_given,
                         final_similarity_when,
                           final_similarity_then,
                           weight_given=1.0,weight_when=1.0,weight_then=1.0):
    return (final_similarity_given * weight_given + final_similarity_when * weight_when + final_similarity_then * weight_then) / (weight_given + weight_when + weight_then)

In [489]:
def local_satisfaction(then_local_similarity_result):
    # aqui eu uso o coeficiente de jaccard para verificar se cada clausula da expressao é satisfeita ou nao
    # quando satisfeita o coeficiente de jaccard é >0 (v>0), logo recebe o valor 1,
    # quando nao satisfeita o coeficiente de jaccard é == 0 (v==0), ele permacenecerá com o valor 0
    return {k: 1 if v > 0 else 0 for k, v in then_local_similarity_result.items()}

In [490]:


desired_scenario=get_desired_scenario()

then_dnf1=to_dnf_expression(get_Then(desired_scenario))

print("Desired Then DNF: ", then_dnf1)
print("")

then_leafs1=get_leafs(then_dnf1)

print("Leafs Then 1: ", then_leafs1)

then_variables1=extract_variables(then_leafs1)

print("Variables Then 1: ", then_variables1)

print("--")

adaptation_effect = "height == 1 AND distance_target == 3" ## estou chamando de then, mas é o contexto alcancado

adaptation_effect=to_dnf_expression(adaptation_effect)

print("Adaptation Effect DNF: ", adaptation_effect)
print("")

then_leafs2=get_leafs(adaptation_effect)

print("Leafs Adaptation Effect: ", then_leafs2)
print("")

then_variables2=extract_variables(then_leafs2)

print("Variables Adaptation Effect: ", then_variables2)
print("")



pair_leaves_then=pair_leaves(then_leafs1, then_leafs2)


print("Pair Leaves Then: ", pair_leaves_then)
print("")


then_local_similarity_result={}

   

for pair in pair_leaves_then:
    then_local_similarity_result[pair]=local_similarity(pair[0], pair[1], load_variable_ranges())


print("Then Local Similarity Result: ", then_local_similarity_result)
print("")

local_satisfaction=local_satisfaction(then_local_similarity_result)
print(local_satisfaction)

zero_keys = [k for k, v in local_satisfaction.items() if v == 0]

for key, value in local_satisfaction.items():
    if value == 0:
        leaf1 = key[0]
        leaf2 = key[1]
        var1, op1, val1 = parse_expression(leaf1)
        var2, op2, val2 = parse_expression(leaf2)
        var_info = load_variable_ranges().get(var1)
        min_val = var_info['min_value']
        max_val = var_info['max_value']
        delta_val=abs(float(val2) - float(val1))
        error=delta_val/(max_val-min_val)
        local_satisfaction[key]={"error":error}
    else:
        local_satisfaction[key]={"error":0}
        
print("Local Satisfaction: ", local_satisfaction)       
    

global_similarity_then=global_satisfaction(local_satisfaction)


print("Global Similarity Then: ", global_similarity_then)
print("")
    

Desired Then DNF:  height == 0 AND distance_target <= 5

Leafs Then 1:  ['height == 0', 'distance_target <= 5']
Variables Then 1:  ['height', 'distance_target']
--
Adaptation Effect DNF:  height == 1 AND distance_target == 3

Leafs Adaptation Effect:  ['height == 1', 'distance_target == 3']

Variables Adaptation Effect:  ['height', 'distance_target']

Pair Leaves Then:  [('height == 0', 'height == 1'), ('distance_target <= 5', 'distance_target == 3')]

Then Local Similarity Result:  {('height == 0', 'height == 1'): 0.0, ('distance_target <= 5', 'distance_target == 3'): 0.2}

{('height == 0', 'height == 1'): 0, ('distance_target <= 5', 'distance_target == 3'): 1}
Local Satisfaction:  {('height == 0', 'height == 1'): {'error': 0.01}, ('distance_target <= 5', 'distance_target == 3'): {'error': 0}}
Global Similarity Then:  0.995



In [491]:
var_info
min_val = var_info['max_value']
min_val

100

In [492]:
load_variable_ranges

<function __main__.load_variable_ranges(file_path='variables_config.json')>

In [493]:
zero_keys
for 

SyntaxError: invalid syntax (3407229389.py, line 2)

In [ ]:
zero_keys[0][0]

'height == 0'

In [ ]:
# # Cria o DataFrame
# df_results = pd.DataFrame(results_to_df).sort_values(by="Scenario Similarity", ascending=False)

# df_results